In [ ]:
import folium
from folium import plugins
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers as eshelper
from elasticsearch_dsl import Search, Q

In [ ]:
client = Elasticsearch(hosts='http://')

In [ ]:
analysis = [a for a in Search(using=client, index='analysis').scan()]

restaurant_analysis = {a.id: a for a in analysis}

In [ ]:
lyon = [(r, restaurant_analysis.get(r.meta.id)) for r in Search(using=client, index='restaurants').query('match', city='Genève').scan() if restaurant_analysis.get(r.meta.id)]
len(lyon)

In [ ]:
lat = []
lng = []
values = []
for r, analyse in lyon:
    print(r)
    try:
        print(analyse.total["Matières grasses"])
        lat.append(r.lat)
        lng.append(r.lng)
        values.append(analyse.total["Matières grasses"].quantity)
    except KeyError:
        pass

In [ ]:
plt_values = np.array(values)

In [ ]:
plt_values.max() - plt_values.min()

In [ ]:
plt_values = plt_values[np.abs(plt_values - plt_values.mean()) / plt_values.std() < 3]

In [ ]:
plt_values.max() - plt_values.min()

In [ ]:
plt_values = 1 * (plt_values - plt_values.min()) / (plt_values.max() - plt_values.min())

In [ ]:
plt.plot(plt_values)
plt.show()

In [ ]:
map_ = folium.Map(location=[46.599505, 3.480752], zoom_start=6)

In [ ]:
plugins.HeatMap(zip(lat, lng, plt_values), radius = 20, max_val=1).add_to(map_)

In [ ]:
map_.save('restaurants.html')